<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lc0/jupyter-notebooks/blob/master/hacktoberfest/find_a_contributor.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lc0/jupyter-notebooks/blob/master/hacktoberfest/find_a_contributor.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [1]:
import requests

In [2]:
# TODO: validate query before sending to API

github_org_query = """
query getMembers($org_name: String!, $limit: Int!, $cursor: String) {
  organization(login: $org_name) {
    membersWithRole(first: $limit, after: $cursor) {
      totalCount
      edges {
        cursor
        node {
          name
          login
          avatarUrl
          location
        }
      }
    }
  }
}
"""

# TODO: validate the generator as well
gq_generator = lambda result: (member for member in result['data']['organization']['membersWithRole']['edges'])

The query itself could be debugged at [Github's query explorer](https://developer.github.com/v4/explorer/), that supports autocomplete and allows to iterate on query pretty quickly

In [3]:
import os

from typing import Dict

class GithubAPI:
    _github_api_url = 'https://api.github.com/graphql'
    
    def __init__(self, token=None):
        if not token:
            try:
                token = os.environ['GITHUB_TOKEN']
            except KeyError:
                raise ValueError("Github token expected. "
                                 "Please pass as an argument or provide in env variable GITHUB_TOKEN.")

        self._token = token
        self._headers = {"Authorization": f"token {self._token}"}
            
            
    def _query_api(self, query: str, variables: Dict[str, str]):
        request = requests.post(self._github_api_url, 
                                json={'query': query, 'variables': variables}, 
                                headers=self._headers)
        if request.status_code == 200:
            return request.json()
        else:
            raise Exception(f"Query failed with status code {request.status_code}: {request.json()}")
            
    def query(self, query: str, variables: Dict[str, str], generator):
        cursor = None
        
        # TODO: add tqdn
        result_list = []
        while True:
            if cursor:
                if 'cursor' in variables and variables['cursor'] == cursor:
                    break
                variables['cursor'] = cursor
                
            results = self._query_api(query, variables)
            
            for item in generator(results):
                result_list.append(item['node'])
                cursor = item['cursor']
            
        return result_list

In order to get a personal access token, please go to the page: [Personal access tokens](https://github.com/settings/tokens).

For this application, I only put `read:org`.

Now we just need to set our token to env with 
```
os.environ['GITHUB_TOKEN'] = 'YOUR-TOKEN'
```

In [4]:
from getpass import getpass

token = getpass('Please enter your Github token: ')

github_api = GithubAPI(token)

Please enter your Github token: ········


In [5]:
params = {
    'org_name': 'openmined',
    'limit': 5
}
results = github_api._query_api(github_org_query, params)
results

{'data': {'organization': {'membersWithRole': {'totalCount': 188,
    'edges': [{'cursor': 'Y3Vyc29yOnYyOpHNb0o=',
      'node': {'name': 'Jeff',
       'login': 'Agent007',
       'avatarUrl': 'https://avatars2.githubusercontent.com/u/28490?v=4',
       'location': 'San Francisco Bay Area'}},
     {'cursor': 'Y3Vyc29yOnYyOpHNp-w=',
      'node': {'name': 'Swaroop C H',
       'login': 'swaroopch',
       'avatarUrl': 'https://avatars0.githubusercontent.com/u/42988?v=4',
       'location': 'San Francisco, USA'}},
     {'cursor': 'Y3Vyc29yOnYyOpHNqnE=',
      'node': {'name': 'Nate Gadzhibalaev',
       'login': 'xnutsive',
       'avatarUrl': 'https://avatars1.githubusercontent.com/u/43633?v=4',
       'location': 'San Francisco'}},
     {'cursor': 'Y3Vyc29yOnYyOpHOAAHFmA==',
      'node': {'name': 'Kevin Mader',
       'login': 'kmader',
       'avatarUrl': 'https://avatars2.githubusercontent.com/u/116120?v=4',
       'location': 'Zurich, Switzerland'}},
     {'cursor': 'Y3Vyc29yOnYyO

## Iterate entire result

First, since graphql does not support flattering results, we added a bit hacky generator `gq_generator`

In [6]:
for item in gq_generator(results):
    print(item['node'])

{'name': 'Jeff', 'login': 'Agent007', 'avatarUrl': 'https://avatars2.githubusercontent.com/u/28490?v=4', 'location': 'San Francisco Bay Area'}
{'name': 'Swaroop C H', 'login': 'swaroopch', 'avatarUrl': 'https://avatars0.githubusercontent.com/u/42988?v=4', 'location': 'San Francisco, USA'}
{'name': 'Nate Gadzhibalaev', 'login': 'xnutsive', 'avatarUrl': 'https://avatars1.githubusercontent.com/u/43633?v=4', 'location': 'San Francisco'}
{'name': 'Kevin Mader', 'login': 'kmader', 'avatarUrl': 'https://avatars2.githubusercontent.com/u/116120?v=4', 'location': 'Zurich, Switzerland'}
{'name': 'Kory', 'login': 'korymath', 'avatarUrl': 'https://avatars0.githubusercontent.com/u/178099?v=4', 'location': 'Edmonton, Alberta, Canada'}


## Query organizations

Shortcut if you are only interested to get data, not in me building up to a solution.

In [7]:
ORGANIZATION = 'openmined'

params = {
    'org_name': ORGANIZATION,
    'limit': 50
}

try:
    github_api = GithubAPI(token)
except NameError:
    token = getpass('Please enter your Github token: ')
    github_api = GithubAPI(token)


members = github_api.query(github_org_query, params, gq_generator)

In [8]:
for member in members:
    if member['location'] and any(candidate in member['location'].lower() for candidate in ['germany', 'munich']):
        print(member)

{'name': 'Sergii Khomenko', 'login': 'lc0', 'avatarUrl': 'https://avatars1.githubusercontent.com/u/775466?v=4', 'location': 'Munich'}
{'name': 'Axel Hodler', 'login': 'axelhodler', 'avatarUrl': 'https://avatars1.githubusercontent.com/u/1504952?v=4', 'location': 'Stuttgart, Germany'}
{'name': 'Andreas Offenhaeuser', 'login': 'anoff', 'avatarUrl': 'https://avatars1.githubusercontent.com/u/7142618?v=4', 'location': 'Germany, Stuttgart'}
{'name': 'Stefan Heidekrüger', 'login': 'heidekrueger', 'avatarUrl': 'https://avatars1.githubusercontent.com/u/9562632?v=4', 'location': 'Munich, Germany'}
{'name': 'Agustín Vargas Toro', 'login': 'Fustincho', 'avatarUrl': 'https://avatars2.githubusercontent.com/u/11200279?v=4', 'location': 'Germany'}


In [9]:
from IPython.core.display import display, HTML

for member in members:
    if member['location'] and any(candidate in member['location'].lower() for candidate in ['germany', 'munich']):
        template = HTML(f"""
<a href='https://github.com/{member['login']}'><img width="83" src='{member['avatarUrl']}'></a> 
<a href='https://github.com/{member['login']}?tab=overview&from=2018-01-01&to=2019-10-31&org={ORGANIZATION}'>Contributions</a>
""")
        display(template)